In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import plotly.express as px
import hdbscan
from sklearn.cluster import KMeans

In [2]:
file_path = 'data//text_all.jsonl'  
data = pd.read_json(file_path, lines=True)


labels_map = {
    0: "pozytywny wydźwięk",
    1: "neutralny wydźwięk",
    2: "negatywny wydźwięk",
    3: "mowa nienawiści",
    '0': "pozytywny wydźwięk",
    '1': "neutralny wydźwięk",
    '2': "negatywny wydźwięk",
    '3': "mowa nienawiści"
}

data["label"] = data["label"].replace(labels_map)
data.head()


,text,label
0,"@USER Nic, nic,nic nieważne, jutro albo w najb...",neutralny wydźwięk
1,@USER Kibic @USER odpowiada @USER i @USER na k...,neutralny wydźwięk
2,Mówi że stare rapsy są całkiem niezle,neutralny wydźwięk
3,"@USER @USER Zaległości były, ale ważne czy był...",neutralny wydźwięk
4,@USER @USER Oby nie spierdolil na północ,negatywny wydźwięk


In [3]:
unique_labels = data['label'].explode().unique()
print(unique_labels)

['neutralny wydźwięk' 'negatywny wydźwięk' 'pozytywny wydźwięk'
 'mowa nienawiści']


In [4]:
data = data[data['label'].apply(lambda x: len(x) > 0)] 
data['label'] = data['label'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)

# TfidfVectorizer

In [5]:
vectorizer = TfidfVectorizer()
tfidf_embeddings = vectorizer.fit_transform(data['text']).toarray()

In [6]:
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(tfidf_embeddings)

tsne_df = pd.DataFrame(tsne_results, columns=['x', 'y'])
tsne_df['label'] = data['label']

fig = px.scatter(tsne_df, x='x', y='y', color='label', 
                 title='Wizualizacja osadzeń TF-IDF przy użyciu t-SNE',
                 labels={'label': 'Wydźwięk'},
                 hover_name=tsne_df['label'],
                 color_discrete_sequence=px.colors.qualitative.Vivid)

fig.update_traces(marker=dict(size=10),
                  selector=dict(mode='markers'))

fig.show()

In [19]:
vectorizer = TfidfVectorizer()
tfidf_embeddings = vectorizer.fit_transform(data['text']).toarray()

hdbscan_model_tfidf = hdbscan.HDBSCAN(min_cluster_size=2)
labels_tfidf = hdbscan_model_tfidf.fit_predict(tfidf_embeddings)

data['cluster_tfidf'] = labels_tfidf

tsne_tfidf = TSNE(n_components=2, random_state=42)
tsne_results_tfidf = tsne_tfidf.fit_transform(tfidf_embeddings)

tsne_df_tfidf = pd.DataFrame(tsne_results_tfidf, columns=['x', 'y'])
tsne_df_tfidf['cluster'] = data['cluster_tfidf']

fig_tfidf = px.scatter(tsne_df_tfidf, x='x', y='y', color='cluster',
                       title='Wizualizacja klastrów z HDBSCAN dla TF-IDF',
                       labels={'cluster': 'Klaster'},
                       color_discrete_sequence=px.colors.qualitative.Vivid)

fig_tfidf.show()


In [20]:
tfidf_embeddings = np.array(tfidf_embeddings)

n_clusters = 4

kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(tfidf_embeddings)

data['kmeans_cluster'] = kmeans.labels_


In [21]:
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(tfidf_embeddings)

tsne_df = pd.DataFrame(X_tsne, columns=['x', 'y'])
tsne_df['kmeans_cluster'] = data['kmeans_cluster']

fig = px.scatter(tsne_df, x='x', y='y', color='kmeans_cluster',
                 title='Wizualizacja klastrów z K-means dla TF-IDF',
                 labels={'kmeans_cluster': 'Klastry'}, 
                 color_discrete_sequence=px.colors.qualitative.Vivid)

fig.show()


# Universal Sentence Encoder

In [10]:
import tensorflow_hub as hub

model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(model_url)

def get_use_embeddings(texts, model):
    return model(texts).numpy()

use_embeddings = get_use_embeddings(data['text'].tolist(), model)

In [11]:
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_results = tsne.fit_transform(use_embeddings)


In [12]:
tsne_df = pd.DataFrame(tsne_results, columns=['x', 'y'])
tsne_df['label'] = data['label']

fig = px.scatter(tsne_df, x='x', y='y', color='label',
                 title='Wizualizacja t-SNE osadzeń Universal Sentence Encoder',
                 labels={'label': 'Wydźwięk'},
                 color_discrete_sequence=px.colors.qualitative.Pastel)

fig.show()

In [22]:
hdbscan_model_use = hdbscan.HDBSCAN(min_cluster_size=20)
labels_use = hdbscan_model_use.fit_predict(use_embeddings)

data['cluster_use'] = labels_use

tsne_use = TSNE(n_components=2, random_state=42)
tsne_results_use = tsne_use.fit_transform(use_embeddings)

tsne_df_use = pd.DataFrame(tsne_results_use, columns=['x', 'y'])
tsne_df_use['cluster'] = data['cluster_use']

fig_use = px.scatter(tsne_df_use, x='x', y='y', color='cluster',
                     title='Wizualizacja klastrów z HDSCAN dla USE',
                     labels={'cluster': 'Klaster'},
                     color_discrete_sequence=px.colors.qualitative.Pastel)

fig_use.show()

In [23]:
use_embeddings = np.array(use_embeddings)
n_clusters = 4

kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(use_embeddings)

data['kmeans_cluster'] = kmeans.labels_


In [24]:
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(use_embeddings)

tsne_df = pd.DataFrame(X_tsne, columns=['x', 'y'])
tsne_df['kmeans_cluster'] = data['kmeans_cluster']

fig = px.scatter(tsne_df, x='x', y='y', color='kmeans_cluster',
                 title='Wizualizacja klastrów z K-means dla USE',
                 labels={'kmeans_cluster': 'Klastry'},
                 color_discrete_sequence=px.colors.qualitative.Pastel)

fig.show()
